In [2]:
#Daniel Dominguez Azamar - Introduccion a la Inteligencia Artificial
import matplotlib.pyplot as plt
import numpy as np

#Referencia : matplotlib.org/stable/users/explain/animations/animations.html#sphx-glr-users-explain-animations-animations-py
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [3]:
limx = [-20, 20]
limy = [-20, 20]

#Mismo codigo de Robot_algoritmo_busqueda de Github
#Nota : Mismo codigo que BBP y A*
class Obstaculo():
    def __init__(self, limxMax, limxMin ,limyMax, limyMin):
        self.limites = [limxMax, limxMin ,limyMax, limyMin]
    def in_collision(self, x, y):
        return self.limites[0]>= x >= self.limites[1] and self.limites[2]>= y >= self.limites[3]

class Obstaculo_circulo():
    def __init__(self, centerx, centery, radius):
        self.center = [centerx, centery]
        self.radius = radius
    def in_collision(self, x, y):
        d = np.sqrt((x-self.center[0])**2 + (y-self.center[1])**2)
        return d <= self.radius

class Obstaculo_triangulo():
    def __init__(self, v1, v2, v3):
        self.vertices = [v1, v2, v3]
    
    def in_collision(self, x, y):
        if 5<= x <= 15 and -15 <= y <= -10:
            if y >= -15 and y <=-10 and x <=10 + (y+15) and x >=10 - (y+15):
                return True
        return False

    def dibujar_t(self):
        x_coords = [v[0] for v in self.vertices] + [self.vertices[0][0]]
        y_coords = [v[1] for v in self.vertices] + [self.vertices[0][1]]
        return x_coords, y_coords

In [4]:
class Nodo():
    def __init__(self, pos=[0,0], papa=None):
        self.pos = pos
        self.papa = papa
        self.h = 0 
        
    def __lt__(self, other):
        return self.h < other.h

    def genera_hijos(self, obstaculos):
        hijos = []
        movimientos = [[0, 1], [1, 0], [0, -1], [-1, 0]]
        
        for mov in movimientos:
            pos_nueva = [self.pos[0] + mov[0], self.pos[1] + mov[1]]
            
            if pos_nueva[0] < limx[0] or pos_nueva[0] > limx[1] or \
               pos_nueva[1] < limy[0] or pos_nueva[1] > limy[1]:
                continue
            
            choque = False
            for ob in obstaculos:
                if ob.in_collision(pos_nueva[0], pos_nueva[1]):
                    choque = True
                    break
            
            if not choque:
                hijos.append(Nodo(pos_nueva, self))
        return hijos

    def calcular_h(self, goalpos):
        self.h = abs(goalpos[0] - self.pos[0]) + abs(goalpos[1] - self.pos[1])

    def greedy(self, goalpos, obstaculos, historial_expansion):
        self.calcular_h(goalpos)
        
        open_list = [self]
        closed_set = set() 
        
        while len(open_list) > 0:
            open_list.sort(key=lambda x: x.h)
            nodo_actual = open_list.pop(0)

            historial_expansion.append(nodo_actual.pos)
            
            if nodo_actual.pos == goalpos:
                path = []
                curr = nodo_actual
                while curr:
                    path.append(curr.pos)
                    curr = curr.papa
                return path[::-1]
            
            closed_set.add(tuple(nodo_actual.pos))
            
            hijos = nodo_actual.genera_hijos(obstaculos)
            
            for hijo in hijos:
                if tuple(hijo.pos) in closed_set:
                    continue
                
                hijo.calcular_h(goalpos)
                
                en_open = False
                for existing_node in open_list:
                    if existing_node.pos == hijo.pos:
                        en_open = True
                        break
                
                if not en_open:
                    open_list.append(hijo)
                    
        return None

In [5]:
obstaculos = []
obstaculos.append(Obstaculo(-10, -15, 5, -5)) 
obstaculos.append(Obstaculo_circulo(10, 5, 3))
obstaculos.append(Obstaculo_triangulo((5, -10), (15, -10), (10, -15)))

start_pos = [0, 0]
goal_pos = [15, -12] 

historial_animacion = []
raiz = Nodo(start_pos)

camino_final = raiz.greedy(goal_pos, obstaculos, historial_animacion)

fig, ax = plt.subplots(figsize=(7,7))
ax.set_xlim(limx[0], limx[1])
ax.set_ylim(limy[0], limy[1])
ax.grid(True, alpha=0.3)

for obs in obstaculos:
    if isinstance(obs, Obstaculo_circulo):
        ax.add_artist(plt.Circle(obs.center, obs.radius, color='black', fill=False))
    elif isinstance(obs, Obstaculo_triangulo):
        tx, ty = obs.dibujar_t()
        ax.plot(tx, ty, color='green', linewidth=2)
    else:
        rect = plt.Rectangle((obs.limites[1], obs.limites[3]), 
                             obs.limites[0]-obs.limites[1], 
                             obs.limites[2]-obs.limites[3], 
                             fill=False, color='red', linewidth=2)
        ax.add_artist(rect)

expansion_plot, = ax.plot([], [], 'b.', markersize=4, label='Nodos Evaluados', alpha=0.5)
path_plot, = ax.plot([], [], 'r-', linewidth=3, label='Camino Encontrado')
head_plot, = ax.plot([], [], 'yo', markersize=6, markeredgecolor='black') 

ax.plot(start_pos[0], start_pos[1], 'go', label='Inicio', markersize=8)
ax.plot(goal_pos[0], goal_pos[1], 'mx', label='Meta', markersize=8)
ax.legend(loc='upper left')

def init():
    expansion_plot.set_data([], [])
    path_plot.set_data([], [])
    head_plot.set_data([], [])
    return expansion_plot, path_plot, head_plot

def update(frame):
    if frame < len(historial_animacion):
        data = np.array(historial_animacion[:frame+1])
        expansion_plot.set_data(data[:, 0], data[:, 1])
        head_plot.set_data([data[-1, 0]], [data[-1, 1]])
    else:
        data = np.array(historial_animacion)
        expansion_plot.set_data(data[:, 0], data[:, 1])
        if camino_final:
            path_arr = np.array(camino_final)
            path_plot.set_data(path_arr[:, 0], path_arr[:, 1])
            head_plot.set_data([path_arr[-1, 0]], [path_arr[-1, 1]])

    return expansion_plot, path_plot, head_plot

total_frames = len(historial_animacion) + 20 
anim = FuncAnimation(fig, update, frames=total_frames, init_func=init,
                     interval=50, blit=True)

plt.close()
HTML(anim.to_jshtml())

In [ ]:
#Referencias : https://matplotlib.org/stable/api/animation_api.html